# Dataset creation & Model Testing

In [1]:
# env imports
import sys
import os
import logging
import numpy as np
import random
import pandas as pd
import torch
from tqdm import tqdm

import rdkit.Chem as Chem
from rdkit.Chem import AllChem
from Bio.PDB import PDBParser
import torchmetrics
from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader

Add tankbind directory to System path:

In [2]:
tankbind_src_folder_path = "./tankbind/"
sys.path.insert(0, tankbind_src_folder_path)

In [3]:
# imports from tankbind
from feature_utils import get_protein_feature, get_clean_res_list, extract_torchdrug_feature_from_mol, get_canonical_smiles
from utils import construct_data_from_graph_gvp, evaulate_with_affinity, evaulate
from model import get_model
from generation_utils import get_LAS_distance_constraint_mask, get_info_pred_distance, write_with_new_coords
from metrics import print_metrics, myMetric

In [4]:
dataset_path = f"dataset_10"
df = pd.read_csv(f'{dataset_path}/Mcule_10000.csv', index_col=0)

Get the protein names:

In [ ]:
def extract_protein_names(file_path):
    protein_names = []
    with open(file_path, 'r') as file:
        for line in file:
            # Get the base name without directory path
            base_name = os.path.basename(line.strip())
            # Split the base name to get the protein name without extension
            protein_name = os.path.splitext(base_name)[0]
            protein_names.append(protein_name)
    return protein_names

file_path = 'pdb_dataset.ds'
protein_names = extract_protein_names(file_path)

# print(protein_names[0:5]. len(protein_names))

Randomly select 10% of the dataset:

In [ ]:
n = len(protein_names) * 0.1
sampled_protein_names = random.sample(protein_names, n)

# print(sampled_protein_names[0:5], len(sampled_protein_names))

#### Get protein features:

In [ ]:
def process_proteins(protein_names, pdb_directory):
    parser = PDBParser(QUIET=True)
    protein_dict = {}

    for proteinName in protein_names:
        try:
            proteinFile = f"{pdb_directory}/{proteinName}.pdb"
            s = parser.get_structure(proteinName, proteinFile)
            res_list = list(s.get_residues())
            clean_res_list = get_clean_res_list(res_list, ensure_ca_exist=True)
            protein_dict[proteinName] = get_protein_feature(clean_res_list)
        except Exception as e:
            print(f"Error processing {proteinName}: {e}")

    return protein_dict

pdb_directory = "PDB_files"  # Directory containing PDB files

protein_dict = process_proteins(sampled_protein_names, pdb_directory)

# torch.save(protein_dict, 'protein_dict.pt')   # execute only in first notebook run

In [ ]:
protein_names_new = list(protein_dict.keys())  # updated protein names after processing, in case some proteins failed to process

# print(len(protein_names_new))

#### Create protein data frame:

In [ ]:
info = []
for protein_name in protein_names_new:
    for i, line in tqdm(df.iterrows(), total=len(df)):
        smiles = line['smiles']
        compund_name = ''
        protein_name = protein_name
        com = ",".join([str(x.round(3)) for x in protein_dict[protein_name][0].mean(axis=0).numpy()])
        info.append([protein_name, compund_name, smiles, "protein_center", com])

info = pd.DataFrame(info, columns=['protein_name', 'compound_name', 'smiles', 'pocket_name', 'pocket_com'])

# torch.save(info, 'data.pt')   # execute only in first notebook run

### Construct Dataset

In [5]:
torch.set_num_threads(1)

In [6]:
class MyDataset_VS(Dataset):
    def __init__(self, root, data=None, protein_dict=None, molecule_dict=None, proteinMode=0, compoundMode=1,
                 pocket_radius=20, shake_nodes=None,
                 transform=None, pre_transform=None, pre_filter=None):
        self.data = data
        self.protein_dict = protein_dict
        self.molecule_dict = molecule_dict
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data = torch.load(self.processed_paths[0])
        self.protein_dict = torch.load(self.processed_paths[1])
        self.molecule_dict = torch.load(self.processed_paths[2])
        self.proteinMode = proteinMode
        self.pocket_radius = pocket_radius
        self.compoundMode = compoundMode
        self.shake_nodes = shake_nodes

    @property
    def processed_file_names(self):
        return ['data.pt', 'proteins.pt', 'molecules.pt']

    def process(self):
        # Save data and protein dictionary
        torch.save(self.data, self.processed_paths[0])
        torch.save(self.protein_dict, self.processed_paths[1])
        torch.save(self.molecule_dict, self.processed_paths[2])

    def len(self):
        return len(self.data)

    def get(self, idx):
        line = self.data.iloc[idx]
        smiles = line['smiles']
        pocket_com = line['pocket_com']
        pocket_com = np.array(pocket_com.split(",")).astype(float) if isinstance(pocket_com, str) else pocket_com
        pocket_com = pocket_com.reshape((1, 3))
        use_whole_protein = line.get('use_whole_protein', False)

        protein_name = line['protein_name']
        protein_data = self.protein_dict.get(protein_name)
        
        if protein_data is None:
            raise ValueError(f"Protein {protein_name} not found in pre-calculated protein dictionary")

        protein_node_xyz, protein_seq, protein_node_s, protein_node_v, protein_edge_index, protein_edge_s, protein_edge_v = protein_data

        # Load precomputed molecular features
        molecule_data = self.molecule_dict.get(smiles)
        if molecule_data is None:
            raise ValueError(f"SMILES {smiles} not found in precomputed molecular dictionary")
        
        coords, compound_node_features, input_atom_edge_list, input_atom_edge_attr_list, pair_dis_distribution = self.molecule_dict[smiles]

        data, input_node_list, keepNode = construct_data_from_graph_gvp(
            protein_node_xyz, protein_seq, protein_node_s, protein_node_v, 
            protein_edge_index, protein_edge_s, protein_edge_v,
            coords, compound_node_features, input_atom_edge_list, input_atom_edge_attr_list,
            pocket_radius=self.pocket_radius, use_whole_protein=use_whole_protein, includeDisMap=True,
            use_compound_com_as_pocket=False, chosen_pocket_com=pocket_com, compoundMode=self.compoundMode
        )
        data.compound_pair = pair_dis_distribution.reshape(-1, 16)
        
        return data

In [9]:
# create dataset dirrectory --> only on first run!
# os.system(f"rm -r {dataset_path}") 
# os.system(f"mkdir -p {dataset_path}") 

Precalculate the molecule_dict s.t. the dataset creation is no longer dependent on torchdrug:

In [ ]:
# execute only in first notebook run
smiles_list = info['smiles'].to_list()
smiles_set = set(smiles_list)

molecule_dict = {}
for molecule in smiles_set:
    smiles = get_canonical_smiles(molecule)
    mol = Chem.MolFromSmiles(smiles)
    mol.Compute2DCoords()
    molecule_dict[molecule] = extract_torchdrug_feature_from_mol(mol, has_LAS_mask=True)

torch.save(molecule_dict, 'dataset_10/processed/molecules.pt')   

In [ ]:
# only excute after running the code above once (& only if needed), otherwise FileNotFoundError
data = torch.load(f'{dataset_path}/processed/data.pt')
protein_dict = torch.load(f'{dataset_path}/processed/proteins.pt')
molecule_dict = torch.load(f'{dataset_path}/processed/molecules.pt')

#### Create dataset instance:

In [7]:
# dataset = MyDataset_VS(root=dataset_path, data=info, protein_dict=protein_dict, molecule_dict=molecule_dict) # only on first run, otherwise execute line below
dataset = MyDataset_VS(root=dataset_path)

### Model Testing

In [ ]:
batch_size = 5 # higher batchsize possible only if enough memmory is available (eg.: 10)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

logging.basicConfig(level=logging.INFO)

model = get_model(0, logging, device)

# load self-dock model
modelFile = "./model/self_dock.pt"

model.load_state_dict(torch.load(modelFile, map_location=device))
_ = model.eval()

data_loader = DataLoader(dataset, batch_size=batch_size, follow_batch=['x', 'y', 'compound_pair'], shuffle=False, num_workers=0) # changed num_workers from 8 to 0, due to multiprocessing error
affinity_pred_list = []
for data in tqdm(data_loader):
    if data.dis_map.shape[0] < 50000: # to filter out proteins with dismap size > 10000 -> 50000 due to batch size 5, can be adjusted as needed (if it's too big, it will cause a memory error)
        data = data.to(device)
        y_pred, affinity_pred = model(data)
        affinity_pred_list.append(affinity_pred.detach().cpu())
    else:
        affinity_pred_list.append(torch.zeros(5, 1))

affinity_pred_list = torch.cat(affinity_pred_list)

In [ ]:
info = dataset.data
info['affinity'] = affinity_pred_list

In [ ]:
info.to_csv(f"{dataset_path}/result_info.csv")

### Choose compound with highest affinity score for each protein:

In [9]:
info = pd.read_csv(f'{dataset_path}/result_info.csv') # only execute if needed

In [10]:
chosen = info.loc[info.groupby('protein_name',sort=False)['affinity'].agg('idxmax')].reset_index()

In [11]:
chosen = chosen[chosen['affinity'] != 0]
chosen_smiles = chosen['smiles'].tolist()

chosen_data = MyDataset_VS(root=dataset_path, data=chosen) 
chosen_data.data = chosen

# print(len(chosen_data))

In [12]:
# get model if previous code not executed in this notebook
device = 'cuda' if torch.cuda.is_available() else 'cpu'

logging.basicConfig(level=logging.INFO)

model = get_model(0, logging, device)

# load self-dock model
modelFile = "./model/self_dock.pt"

model.load_state_dict(torch.load(modelFile, map_location=device))
_ = model.eval()


16:03:52   5 stack, readout2, pred dis map add self attention and GVP embed, compound model GIN


In [13]:
data_loader = DataLoader(chosen_data, batch_size=1, follow_batch=['x', 'y', 'compound_pair'], shuffle=False, num_workers=0)

y_preds = []
tankbind_list = []
old_coords = []
new_coords_list = []

for i, data_with_batch_info in enumerate(tqdm(data_loader)):

    y_pred, affinity_pred = model(data_with_batch_info)

    coords = data_with_batch_info.coords.to(device)
    old_coords.append(coords)
    protein_nodes_xyz = data_with_batch_info.node_xyz.to(device)
    n_compound = coords.shape[0] 
    n_protein = protein_nodes_xyz.shape[0] 
    y_pred = y_pred.reshape(n_protein, n_compound).to(device).detach()
    y_preds.append(y_pred)
    y = data_with_batch_info.dis_map.reshape(n_protein, n_compound).to(device)
    compound_pair_dis_constraint = torch.cdist(coords, coords)

    # Extract SMILES and generate 2D coordinates
    smiles = chosen_smiles[i]  
    mol = Chem.MolFromSmiles(smiles)
    mol.Compute2DCoords()

    # Compute LAS distance constraint mask
    LAS_distance_constraint_mask = get_LAS_distance_constraint_mask(mol).bool()

    # Calculate information
    info = get_info_pred_distance(coords, y_pred, protein_nodes_xyz, compound_pair_dis_constraint,
                                  LAS_distance_constraint_mask=LAS_distance_constraint_mask,
                                  n_repeat=1, show_progress=False)

    # Save to file
    toFile = f'{dataset_path}/chosen/KIBA_tankbind_{i}.sdf'
    new_coords = info.sort_values("loss")['coords'].iloc[0].astype(np.double)
    new_coords_list.append(torch.tensor(new_coords))
    write_with_new_coords(mol, new_coords, toFile)
    
    tankbind_list.append([mol, new_coords])


100%|██████████| 85/85 [08:48<00:00,  6.22s/it]


### Evaluation

In [28]:
target_affinity = np.load("KIBA load & process/target_affinity.npy")

In [29]:
chosen_info = chosen_data.data
chosen_info['new_affinity'] = target_affinity

comp_df = chosen_info['affinity'].compare(chosen_info['new_affinity'])

# manual computaion of MAE
comp_df['Score_diff'] = (chosen_info['affinity'] - chosen_info['new_affinity']).abs()
comp_df_sum = comp_df['Score_diff'].sum() / len(chosen_data)
print("{:.8f}".format(comp_df_sum))

4.01976171


### MSELoss & MAE

In [90]:
def mse_mae_metric(pred_aff, target_aff):
    # Assuming pred_aff and target_aff are already tensors
    criterion = torch.nn.MSELoss()
    with torch.no_grad():
        loss = criterion(pred_aff, target_aff)

    mae = torchmetrics.functional.mean_absolute_error(pred_aff, target_aff)

    return {"loss": loss.item(), "MAE": mae}

In [93]:
chosen_aff_tensor= torch.tensor(chosen_info['affinity'].tolist(), requires_grad=True)
target_aff_tensor = torch.tensor(chosen_info['new_affinity'].tolist())
metrics = mse_mae_metric(chosen_aff_tensor, target_aff_tensor)

In [94]:
print_metrics(metrics)

'loss:19.993, MAE: 4.020'

In [88]:
torch.save(metrics, f'{dataset_path}/MSE_MAE_metrics.pt')